In [8]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import glob
import os 

Upload all geo file , which can be downloaded form (put name of geographic website), upload all the file in github repo and then copy and paste the link below. Important note: please do not change the file name of excel

In [9]:
!git clone https://github.com/hanihaider1/why-my-city-smell-bad-/

fatal: destination path 'why-my-city-smell-bad-' already exists and is not an empty directory.


In [10]:
path = './why-my-city-smell-bad-/'
lst = os.listdir(path)

In [11]:
df= pd.DataFrame()
df_new= pd.DataFrame()

for i in lst:
  url = path+i
  if i[-4:]==".csv":
    if df.empty==True :
      df = pd.read_csv(url)
      df["Pod Name"] = i[:5]
    else:
      df_new= pd.read_csv(url)
      df_new["Pod Name"]= i[:5]
      df= df.append(df_new)

In [12]:
def add_dates_times_columns(df):
  # insert date only, time only, and DateTime Columns
  # adding a filler value x or y that will be replaced later
  df.insert(1, 'date_only', 'x')
  df.insert(2, 'time_only', 'y')
  
  # change name of Time column to datetime for clarity that it is a datetime
  df.rename(columns={'Time' : 'DateTime'}, inplace=True)
  #copy string of DateTime to time and date only columns
  df['date_only'] = df['DateTime']
  df['time_only'] = df['DateTime']

  # convert strings to appropriate data type - DateTime, date, and time
  df['DateTime'] = pd.to_datetime(df['DateTime'], format="%Y-%m-%d %H:%M:%S")
  df['date_only'] = pd.to_datetime(df['date_only']).dt.date
  df['time_only'] = pd.to_datetime(df['time_only']).dt.time

  # resetting index to make it look neat
  df.reset_index(drop=True, inplace=True)



  #added the cardinal point for direction on wind
#making a function to generalise it

def cardinal_direction(df): 
  condition_list = [(df['Wind Direction'] > 0) & (df['Wind Direction'] < 45), (df['Wind Direction'] == 45), (df['Wind Direction'] > 45) & (df['Wind Direction'] <= 90), (df['Wind Direction'] > 90) & (df['Wind Direction'] < 135), (df['Wind Direction'] == 135), (df['Wind Direction'] > 135) & (df['Wind Direction'] <= 180), (df['Wind Direction'] > 180) & (df['Wind Direction'] < 225), (df['Wind Direction'] == 225), (df['Wind Direction'] > 225) & (df['Wind Direction'] <= 270), (df['Wind Direction'] > 270) & (df['Wind Direction'] < 315), (df['Wind Direction'] == 315), (df['Wind Direction'] > 315) & (df['Wind Direction'] <= 360) ]
  choicelist = ['North','North-East', 'East', 'East', 'South-East', 'South', 'South', 'South-West', 'West', 'West', 'North-West', 'North']
  df['Cardinal Direction'] = np.select(condition_list, choicelist) #adding the cardinal direction column
  direction_values=df['Cardinal Direction'].values #to get the values either as array or a series
  df.drop(['Cardinal Direction'], axis=1, inplace=True) # we need to drop the column to relocate to the position we want
  df.insert(15,'Cardinal Direction',value=direction_values)
  return df # to return the dataframe when the function is called



In [13]:
cardinal_direction(df) #call the function
add_dates_times_columns(df)


In [14]:
# Here I sorted by Pod first then by time in terms of priority
df_sorted = df.sort_values(by=['Pod Name','DateTime'])
# reset index to make it look neat
df_sorted.reset_index(drop=True, inplace=True)

In [15]:
df_sorted

,DateTime,date_only,time_only,Temp Avg,Temp Low,Temp High,Hum Avg,Hum Low,Hum High,Baro Avg,Baro Low,Baro High,Windspeed,Gust,Wind Direction,Interval Precip,Leaf Wetness (Minutes),Cardinal Direction,Solar Radiation Avg,Inside Temp Avg,Inside Temp Low,Inside Temp High,Heat Index,Wind Chill,Dew Point,UV Index,Station Voltage,Station Voltage 2,t1ia,t1il,t1ih,t2ia,t2il,t2ih,wetbulb,s2ia,smia,lvl1,lvl2,lvl3,Pod Name
0,2020-09-02 12:00:00,2020-09-02,12:00:00,75.00,75.0,75.0,60.0,60.0,60.0,29.97,29.97,29.98,0.00000,0.0,157.0,0.00,NaN,South,NaN,66.51,65.0,69.0,75.00,75.0,60.2,NaN,6.530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,NaN,NaN,NaN,NaN,NaN,SMRO3
1,2020-09-02 13:00:00,2020-09-02,13:00:00,75.16,75.0,75.2,59.0,58.0,60.0,29.96,29.95,29.98,0.00000,0.0,157.0,0.00,NaN,South,NaN,73.92,69.0,75.0,75.16,75.2,59.8,NaN,6.504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.3,NaN,NaN,NaN,NaN,NaN,SMRO3
2,2020-09-02 14:00:00,2020-09-02,14:00:00,75.41,75.2,75.5,58.0,58.0,58.0,29.94,29.93,29.95,0.00000,0.0,135.0,0.00,NaN,South-East,NaN,74.98,74.0,75.0,75.41,75.4,59.6,NaN,6.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.3,NaN,NaN,NaN,NaN,NaN,SMRO3
3,2020-09-02 15:00:00,2020-09-02,15:00:00,75.32,75.2,75.4,59.0,58.0,59.0,29.91,29.90,29.93,0.00000,0.0,135.0,0.00,NaN,South-East,NaN,74.95,74.0,75.0,75.32,75.3,59.9,NaN,6.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.4,NaN,NaN,NaN,NaN,NaN,SMRO3
4,2020-09-02 16:00:00,2020-09-02,16:00:00,75.47,75.2,75.7,59.0,59.0,59.0,29.89,29.88,29.90,0.00000,0.0,135.0,0.00,NaN,South-East,NaN,74.90,74.0,75.0,75.47,75.5,60.1,NaN,6.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.6,NaN,NaN,NaN,NaN,NaN,SMRO3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23522,2021-07-23 20:00:00,2021-07-23,20:00:00,64.32,63.7,64.7,75.0,70.0,81.0,30.14,30.13,30.15,2.03667,7.2,111.0,0.01,NaN,East,NaN,82.77,82.0,83.0,64.32,64.3,56.3,NaN,6.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.4,NaN,NaN,NaN,NaN,NaN,SMRO7
23523,2021-07-23 21:00:00,2021-07-23,21:00:00,63.51,62.7,64.0,81.0,80.0,83.0,30.13,30.12,30.15,3.15167,9.3,283.0,0.00,NaN,West,NaN,82.28,82.0,83.0,63.51,63.5,57.5,NaN,6.766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.8,NaN,NaN,NaN,NaN,NaN,SMRO7
23524,2021-07-23 22:00:00,2021-07-23,22:00:00,62.94,62.7,63.1,81.0,79.0,83.0,30.15,30.14,30.15,3.17167,9.5,315.0,0.00,NaN,North-West,NaN,82.00,82.0,82.0,62.94,62.9,57.1,NaN,6.685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.4,NaN,NaN,NaN,NaN,NaN,SMRO7
23525,2021-07-23 23:00:00,2021-07-23,23:00:00,62.57,62.0,63.0,80.0,79.0,80.0,30.14,30.14,30.15,4.81667,11.0,317.0,0.00,NaN,North,NaN,82.00,82.0,82.0,62.57,62.6,56.2,NaN,6.603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.7,NaN,NaN,NaN,NaN,NaN,SMRO7


#i think we need to add long lat for the pods 